In [ ]:
import matplotlib
import pandas as pd
import numpy as np
import json
import math
import urllib.request
import dateutil.parser
import dateutil.rrule
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import gc
from textwrap import wrap

matplotlib.rcParams.update({
    'font.size': 13,
    'timezone': 'Europe/London',
    'figure.max_open_warning': False
})

In [ ]:
# Used across all of the plots
dateToday = datetime.datetime.combine(datetime.date.today(), datetime.datetime.min.time())

colourUp = '#f64a8a'
colourDown = '#233067'

resampleFrequency = 900

# Traffic volumes across Tyne and Wear

The data represents numberplates read using ANPR cameras. These are normally used to calculate journey times between two points on the road network, and in turn used to invoke traffic signal strategies or identify problems. No actual numberplate data was used by Newcastle University in preparing these statistics.

Each journey time pair has been disaggregated to the start and end points representing individual cameras. The number of plates read by that camera is then used for these graphs.


In [ ]:
dfPointTs = pd.read_pickle('../cache/recent-traffic-volumes-pd.pkl')
anprLinks = pd.read_pickle('../cache/recent-traffic-volumes-link-metadata-pd.pkl')
anprPoints = pd.read_pickle('../cache/recent-traffic-volumes-point-metadata-pd.pkl')

print('Last data obtained %s' 
    % (np.max(dfPointTs.index).strftime('%d %B %Y %H:%M')))

#anprPoints

In [ ]:
def plotTrafficVolumeTimeseries(historicCutOffDays=28):
    dfTrafficRecent = dfPointTs[dfPointTs.index >= dateToday - pd.Timedelta(days=historicCutOffDays)] \
        .apply(lambda count: count / (900 / 60))
    
    # Sometimes it has stupid numbers, presumably because of a comms drop on the ANPR camera?
    dfTrafficRecent = dfTrafficRecent[dfTrafficRecent < 60]

    groupIndex = -1
    plotIndex = 0
    plotHighway = None

    sortedCameras = sorted(
        dfTrafficRecent.columns,
        key=lambda colName: anprPoints[anprPoints['timeseriesName'] == colName]['highwayDescription'].values[0]
    )

    for timeseriesName in sortedCameras:
        pointMeta = anprPoints[anprPoints['timeseriesName'] == timeseriesName].to_dict(orient='records')[0]
        
        if pointMeta['highwayDescription'] != plotHighway:
            # Start a new plot
            plotHighway = pointMeta['highwayDescription']
            pointsInSystem = list(anprPoints[anprPoints['highwayDescription'] == plotHighway]['timeseriesName'])
            pointsWithData = list(filter(lambda p: p in dfTrafficRecent.columns, pointsInSystem))
            pointsWithDataCount = len(pointsWithData)

            fig, axs = plt.subplots(
                pointsWithDataCount, 1,
                figsize = (18, 1.5 * pointsWithDataCount + 0.75),
                constrained_layout=True
            )
            
            if pointsWithDataCount == 1:
                axs = [axs]
            
            plotIndex = 0
            groupIndex = groupIndex + 1
        
        ax = axs[plotIndex]
        
        ax.set_title(
             ('%s\n⇀ at %s (%s at %s)' % (
                  pointMeta['highwayDescription'],
                  pointMeta['pointDescription'].replace('(BUS)', ''),
                  pointMeta['systemCodeNumber'],
                  pointMeta['end']
             )) if plotIndex == 0 else
             ('⇀ at %s (%s at %s)' % (
                  pointMeta['pointDescription'].replace('(BUS)', ''),
                  pointMeta['systemCodeNumber'],
                  pointMeta['end']
             )),
             loc='left',
             fontdict={ 'horizontalalignment': 'left', 'fontsize': 12 }
        )
        ax.margins(x=0, y=0)

        ax.fill_between(
            dfTrafficRecent[timeseriesName].index,
            dfTrafficRecent[timeseriesName],
            color=colourDown,
            where=dfTrafficRecent.index.to_series().apply(
                 lambda t: t.strftime('%A') not in ['Saturday', 'Sunday']
            ),
            label='Vehicles per minute on weekdays'
        )
        ax.fill_between(
            dfTrafficRecent[timeseriesName].index,
            dfTrafficRecent[timeseriesName],
            color=colourUp,
            where=dfTrafficRecent.index.to_series().apply(
                 lambda t: t.strftime('%A') in ['Saturday', 'Sunday']
            ),
            label='Vehicles per minute on weekend'
        )

        ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=1, byweekday=mdates.MO))
        ax.xaxis.set_tick_params(which='major', pad=15)
        ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))

        if ax == axs[-1]:
            ax.set_xlabel('Date')

            if historicCutOffDays > 75:
                timeLocatorMajor = mdates.AutoDateLocator(minticks=10, maxticks=30)
                conciseZeroFormats = ['', '%Y', '%b', '%d-%b', '%H:%M', '%H:%M']
                conciseOffsetFormats = ['', '%Y', '%b-%Y', '%d-%b-%Y-%b', '%d-%b-%Y', '%d-%b-%Y %H:%M']
                ax.xaxis.set_tick_params(which='major', pad=0)
                ax.xaxis.set_major_locator(timeLocatorMajor)
                ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator=timeLocatorMajor, zero_formats=conciseZeroFormats, offset_formats=conciseOffsetFormats))
            else:
                dataFormatMajor = mdates.DateFormatter('%a %d %b')
                ax.xaxis.set_major_formatter(dataFormatMajor)
                ax.xaxis.set_minor_formatter(mdates.DateFormatter('%d'))
        else:
            ax.xaxis.set_ticklabels([]);

        if ax == axs[0] and groupIndex == 0:
            ax.legend(
                loc='upper right',
                ncol=2,
                fontsize=11,
                frameon=False,
                bbox_to_anchor=(1.0, 1.35)
            )

        plotIndex = plotIndex + 1

    #plt.tight_layout()
    #fig.subplots_adjust(top=2.0)
    plt.show()
    
    gc.collect()

## Daily occupancy of car parks during last 6 weeks

The below plot shows the profile of the car park occupancy within the last month or so, rather than being aggregated to a daily level.

In [ ]:
# Continuous plot of areas for the last N days
plotTrafficVolumeTimeseries(21)